In [0]:
from pyspark.sql import SparkSession

# Initialize Spark Session
spark = SparkSession.builder.appName("PySparkSQLAdvanced").getOrCreate()
print(spark)

In [0]:
# Load the dataset
df = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/shared_uploads/Employee_Salary_Dataset.csv")

# Show the first 5 rows
df.show(5)

+---+----------------+---+------+------+
| ID|Experience_Years|Age|Gender|Salary|
+---+----------------+---+------+------+
|  1|               5| 28|Female|250000|
|  2|               1| 21|  Male| 50000|
|  3|               3| 23|Female|170000|
|  4|               2| 22|  Male| 25000|
|  5|               1| 17|  Male| 10000|
+---+----------------+---+------+------+
only showing top 5 rows



In [0]:
df.createOrReplaceTempView("employees")

In [0]:
result = spark.sql("SELECT * FROM employees WHERE salary > 5000")
result.show()

+---+----------------+---+------+-------+
| ID|Experience_Years|Age|Gender| Salary|
+---+----------------+---+------+-------+
|  1|               5| 28|Female| 250000|
|  2|               1| 21|  Male|  50000|
|  3|               3| 23|Female| 170000|
|  4|               2| 22|  Male|  25000|
|  5|               1| 17|  Male|  10000|
|  6|              25| 62|  Male|5001000|
|  7|              19| 54|Female| 800000|
|  8|               2| 21|Female|   9000|
|  9|              10| 36|Female|  61500|
| 10|              15| 54|Female| 650000|
| 11|               4| 26|Female| 250000|
| 12|               6| 29|  Male|1400000|
| 13|              14| 39|  Male|6000050|
| 14|              11| 40|  Male| 220100|
| 15|               2| 23|  Male|   7500|
| 16|               4| 27|Female|  87000|
| 17|              10| 34|Female| 930000|
| 18|              15| 54|Female|7900000|
| 19|               2| 21|  Male|  15000|
| 20|              10| 36|  Male| 330000|
+---+----------------+---+------+-

In [0]:
result = spark.sql("""
    SELECT Gender, AVG(salary) AS avg_salary
    FROM employees
    GROUP BY Gender
""")
result.show()

+------+------------------+
|Gender|        avg_salary|
+------+------------------+
|Female|2054916.6666666667|
|  Male|2063626.4705882352|
+------+------------------+



In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import sum

# Define a window specification
window_spec = Window.partitionBy("gender").orderBy("salary")

# Add a cumulative salary column
df_with_cumulative = df.withColumn("cumulative_salary", sum("salary").over(window_spec))

# Show the result
df_with_cumulative.show()

+---+----------------+---+------+--------+-----------------+
| ID|Experience_Years|Age|Gender|  Salary|cumulative_salary|
+---+----------------+---+------+--------+-----------------+
| 28|              27| 62|Female|10000000|            1.0E7|
| 33|              20| 55|Female| 1540000|          1.154E7|
|  3|               3| 23|Female|  170000|          1.171E7|
| 26|               3| 22|Female|   20000|          1.173E7|
|  1|               5| 28|Female|  250000|          1.223E7|
| 11|               4| 26|Female|  250000|          1.223E7|
| 29|              19| 54|Female| 5000000|          1.723E7|
| 24|               1| 21|Female|    6000|         1.7236E7|
| 30|               2| 21|Female|    6100|        1.72421E7|
|  9|              10| 36|Female|   61500|        1.73036E7|
| 10|              15| 54|Female|  650000|        1.79536E7|
| 18|              15| 54|Female| 7900000|        2.58536E7|
|  7|              19| 54|Female|  800000|        2.66536E7|
| 16|               4| 2

In [0]:
result = spark.sql("""
    SELECT Id, salary
    FROM employees
    WHERE salary > (SELECT AVG(salary) FROM employees)
""")
result.show()

+---+--------+
| Id|  salary|
+---+--------+
|  6| 5001000|
| 13| 6000050|
| 18| 7900000|
| 21| 6570000|
| 23| 6845000|
| 28|10000000|
| 29| 5000000|
| 34| 9300000|
| 35| 7600000|
+---+--------+

